In [8]:
import re
import torch
import random
import evaluate
import Levenshtein
import pandas as pd 
import numpy as np

from datasets import Dataset
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, EarlyStoppingCallback, BertTokenizerFast, AutoTokenizer

In [ ]:
def predict_entity(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(model.device)

    with torch.no_grad():
        logits = model(**inputs).logits
    predictions = torch.argmax(logits, dim=2).squeeze().tolist()

    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze())
    pred_labels = predictions

    filtered_tokens = [
        token.replace("#", "") 
        for token, label in zip(tokens, pred_labels)
        if label == 1 and token not in tokenizer.all_special_tokens
    ]

    predicted_entity = "".join(filtered_tokens)  # объединяем без пробелов
    return predicted_entity

In [21]:
model_path = "ner_model/checkpoint-2560"

tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")
model = AutoModelForTokenClassification.from_pretrained(model_path)

In [22]:
result = predict_entity("Привет вот мой артикула - JFDBFD-1231", model, tokenizer)

In [23]:
result

'JFDBFD-1231'